In [1]:
!pip install transformers accelerate
!pip install transformers[sentencepiece]
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
!pip install datasets
from datasets import load_metric
%pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
!pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# from google.colab import drive
# drive.mount('/content/drive') ['Corrective','Adaptive','Perfective']
label2id = {'a':0,'p':1,'c':2}

In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/Commit_dataset.csv',encoding='cp1252')
df = df.replace({"3_labels": label2id})
# # print(df)
# test_sample = df.sample(n=3, random_state=1)

In [5]:
df

,commit_id,project,comment,3_labels,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
0,0531b8bff5c14d9504beefb4ad47f473e3a22932,ReactiveX-RxJava,Change hasException to hasThrowable--,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0950c46beda335819928585f1262dfe1dca78a0b,ReactiveX-RxJava,Trying to extend the Scheduler interface accor...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0f92fdd8e6422d5b79c610a7fd8409d222315a49,ReactiveX-RxJava,RunAsync method for outputting multiple values--,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100f571c9a2835d5a30a55374b9be74c147e031f,ReactiveX-RxJava,forEach with Action1 but not Observer--I re-re...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,191f023cf5253ea90647bc091dcaf55ccdce81cc,ReactiveX-RxJava,1.x: Fix Completable swallows- OnErrorNotImple...,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,51e9da224f80254476a7dc446bca817b505381d8,jenkinsci$clearcase-plugin,Use a temporary file to decrease memory consum...,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1789,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,jexp$batch-import,added a more memory efficient structure for st...,1,0,0,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1790,19b650c78a1c76f4fd90274d7f163f863c0d39e4,hdiv$hdiv,Memory and performance optimizations,1,0,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
1791,d7bf95159df37a3d338ca267dddd3d26b38ec37c,casidiablo$persistence,Now it is possible to specify the sqlite open ...,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df = df.rename(columns={'3_labels':'label','comment':'text'})
# le = preprocessing.LabelEncoder()
# le.fit(df.label)
# df['label'] = le.transform(df.label)

In [7]:
train, test = train_test_split(df,train_size=0.70,stratify=df['label'],random_state=1)

In [52]:
train.to_csv('train.csv',encoding='utf_8_sig')

In [53]:
test.to_csv('test.csv',encoding='utf_8_sig')

In [10]:
from openprompt.data_utils import InputExample

dataset = {}
# for split in ['train','test']:
    # dataset[split] = []
dataset['train'] = []
dataset['test'] = []
for index, row in train.iterrows():
# print(row.text)
# print(row['text'])
  input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
  dataset['train'].append(input_example)
for index, row in test.iterrows():
    # print(row.text)
    # print(row['text'])
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)
print(dataset['train'][0])

{
  "guid": null,
  "label": 0,
  "meta": {},
  "text_a": "Trying to extend the Scheduler interface according- to the comments at -19.--",
  "text_b": "",
  "tgt_text": null
}



In [11]:
len(train)

1255

In [12]:
len(test)

538

In [13]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available 

In [14]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)
# mytemplate = SoftTemplate(model=plm, tokenizer=tokenizer, num_tokens=args.soft_token_num, initialize_from_vocab=args.init_from_vocab).from_file(f"scripts/{scriptsbase}/soft_template.txt", choice=args.template_id)
# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' Trying to extend the Scheduler interface according- to the comments at -19.--', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 0}]


In [45]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 3, 26421, 12, 4285, 8, 14890, 52, 3459, 1315, 18, 12, 8, 2622, 44, 3, 4481, 5, 18, 18, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁', 'Trying', '▁to', '▁extend', '▁the', '▁Schedule', 'r', '▁interface', '▁according',

In [46]:
model_inputs = {}
for split in ['train', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

In [47]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))

tokenizing: 1255it [00:01, 751.73it/s]


In [48]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
# myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
#                         label_words=[["Adaptive"], ["Perfective"], ["Corrective"]])
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=3).from_file("/content/drive/MyDrive/knowledgeable_verbalizer.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


##Num of label words for each label: [294, 114, 230]
Parameter containing:
tensor([[[    3, 14808,  1528,  ...,     0,     0,     0],
         [ 4639,     0,     0,  ...,     0,     0,     0],
         [27679,    53,     0,  ...,     0,     0,     0],
         ...,
         [ 1147,  3225,     3,  ...,     0,     0,     0],
         [  954,   635,  1208,  ...,     0,     0,     0],
         [    8,    32,    26,  ...,   172,  2618,  5352]],

        [[ 7710,   757,     0,  ...,     0,     0,     0],
         [28306,   757,  2663,  ...,     0,     0,     0],
         [  626,     0,     0,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[28223,   757,     0,  ...,     0,     0,     0],
         [    3, 15471,     0,  ...,     0,     0,     0],
         [ 1709,  1528,     0,  ...,     0,     0,     0],
        

In [49]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [50]:
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [51]:
# Set up early stopping parameters
best_validation_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
waited_epochs = 0
# Evaluate
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
for epoch in range(10):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss / (step + 1)), flush=True)

    # Validation loop
    validation_loss = 0
    prompt_model.eval()
    with torch.no_grad():
        for val_step, val_inputs in enumerate(validation_dataloader):
            if use_cuda:
                val_inputs = val_inputs.cuda()
            val_logits = prompt_model(val_inputs)
            val_labels = val_inputs['label']
            val_loss = loss_func(val_logits, val_labels)
            validation_loss += val_loss.item()

    validation_loss /= (val_step + 1)
    print("Epoch {}, validation loss: {}".format(epoch, validation_loss), flush=True)

    # Check for early stopping
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        waited_epochs = 0
        # Save the model's state dictionary to a file
        torch.save(prompt_model.state_dict(), "best_model_state_dict.pt")
    else:
        waited_epochs += 1
        if waited_epochs >= patience:
            print("Early stopping at epoch {}.".format(epoch), flush=True)
            break

    prompt_model.train()  # Set the model back to training mode

tokenizing: 538it [00:00, 761.76it/s]


Epoch 0, average loss: 0.0041622135322541
Epoch 0, average loss: 0.01138995600779833
Epoch 0, average loss: 0.029733098163626775


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
class_names = ['Adaptive','Perfective','Corrective']  # Replace num_classes with the number of classes in your dataset
classification_rep = classification_report(alllabels, allpreds, target_names=class_names, digits=4)

print("Classification Report:")
print(classification_rep)